In [1]:
import os, sklearn, openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snss
import numpy as np
import nltk
import pickle

data_dir = os.path.join(os.getcwd(), "Dataset")

In [2]:
music_df = pd.read_csv(os.path.join(data_dir, "Music_Lyrics.csv"), index_col=0)

In [17]:
processed_df = pd.read_csv(os.path.join(data_dir,"emotions_processed_df.csv"))
processed_df = processed_df.rename(columns={"sentiment": "labels", "content": "text"})


In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(processed_df['labels'])
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))
#print(le_name_mapping)

#print both key and value from mapping 
for key, value in le_name_mapping.items():
    print(key, value)

0 anger
1 boredom
2 empty
3 enthusiasm
4 fear
5 fun
6 happiness
7 happy
8 hate
9 love
10 neutral
11 relief
12 sadness
13 worry


In [3]:
model = pickle.load(open("logistic_regression-ovr-c1_emotions.pkl", "rb"))
from sklearn.feature_extraction.text import CountVectorizer
cv = pickle.load(open('cv (selected) emotions.pkl', 'rb')) # load the vectorizer

In [4]:
test = [music_df["lyrics"].iloc[224]]
test

['moon gold love sweetheart say goodbye moonlight kiss share memory light glow tender word whisper promise come true know soon leave long moon glow go sweetheart memory grow cold tarnish moon gold go sweetheart memory grow cold tarnish moon gold']

In [5]:
X_test_bow = cv.transform(test).toarray()

In [25]:
#return results from mapping dictionary
def results(result):
    return le_name_mapping[result[0]]
    

In [26]:
result = model.predict(X_test_bow)
print(results(result))

love


## Transformer

In [27]:
import os, warnings, torch
import numpy as np
import pandas as pd
import transformers
warnings.filterwarnings('ignore')

output_dir = os.path.join(os.getcwd(), "models", "bert-base-uncased")

In [28]:
# Load a trained model and vocabulary that you have fine-tuned
model_class, tokenizer_class, pretrained_weights = (transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased')

model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
def to_check_result(test_encoding):
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    with torch.no_grad():
        output = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
    y = np.argmax(output[0].to("cpu").numpy())
    return y

In [30]:
test_emotions = []
test = [music_df["lyrics"].iloc[224]]

for i in test:
    test_encoding1 = tokenizer(i, padding=True, truncation=True)
    input_ids = torch.tensor(test_encoding1["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding1["attention_mask"]).to(device)
    test_emotions.append(to_check_result(test_encoding1))

In [31]:
print(results(test_emotions))

love
